<a href="https://colab.research.google.com/github/esb-index/Barka-AV/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xarray pandas numpy netCDF4 openpyxl --upgrade

import zipfile, os, xarray as xr, pandas as pd, numpy as np
from google.colab import files

# 1️⃣ Töltsd fel ide a Copernicus ZIP fájlt
uploaded = files.upload()
zip_path = list(uploaded.keys())[0]
extract_dir = "/content/era5_extracted"
os.makedirs(extract_dir, exist_ok=True)

# 2️⃣ Kicsomagolás
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_dir)

# 3️⃣ Keresd meg az adatfájlokat
files_in_zip = os.listdir(extract_dir)
print("📂 Talált fájlok:", files_in_zip)

# 4️⃣ Azonosítsd a hőmérséklet- és csapadékfájlokat
temp_file = [f for f in files_in_zip if "instant" in f][0]
precip_file = [f for f in files_in_zip if "accum" in f][0]

ds_temp = xr.open_dataset(os.path.join(extract_dir, temp_file))
ds_precip = xr.open_dataset(os.path.join(extract_dir, precip_file))

# 5️⃣ Változók automatikus felismerése
var_temp = list(ds_temp.data_vars)[0]
var_precip = list(ds_precip.data_vars)[0]

# Hőmérséklet °C-ra
df_temp = ds_temp[var_temp].to_dataframe().reset_index()
df_temp["t2m"] = df_temp[var_temp] - 273.15

# Csapadék
df_precip = ds_precip[var_precip].to_dataframe().reset_index()

# Idő dimenzió felismerése
time_col = [c for c in df_temp.columns if "time" in c][0]
df_temp["year"] = pd.to_datetime(df_temp[time_col]).dt.year
df_precip["year"] = pd.to_datetime(df_precip[time_col]).dt.year

# Éves aggregálás
agg_temp = df_temp.groupby("year").agg(
    mean_temp=("t2m", "mean"),
    max_temp=("t2m", "max")
).reset_index()

agg_precip = df_precip.groupby("year").agg(
    total_precip=(var_precip, "sum")
).reset_index()

agg = pd.merge(agg_temp, agg_precip, on="year", how="outer")

# Hőhullám-napok (T > 30°C)
df_temp["heatwave_flag"] = (df_temp["t2m"] > 30).astype(int)
heatwave_days = df_temp.groupby("year")["heatwave_flag"].sum().reset_index(name="heatwave_days")

agg = agg.merge(heatwave_days, on="year", how="left").fillna(0)

# 6️⃣ Mentés
out = "ERA5_Hungary_2000_2022_processed.xlsx"
agg.to_excel(out, index=False)
files.download(out)
print("✅ Feldolgozás kész! Eredmény:", out)


Saving d39e64c49d9afe436f96e1d6e3a34e96.zip to d39e64c49d9afe436f96e1d6e3a34e96 (1).zip
📂 Talált fájlok: ['data_stream-oper_stepType-instant.nc', 'data_stream-oper_stepType-accum.nc']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Feldolgozás kész! Eredmény: ERA5_Hungary_2000_2022_processed.xlsx
